<a href="https://colab.research.google.com/github/ushasri999/Multiclass-classification-of-DDoS-attacks-in-IoT-network-using-hybrid-feature-selection-algorithm/blob/main/XGB_34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**34 Classes, 15 features**

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Step 1: Load and filter the dataset
df = pd.read_csv('/content/drive/MyDrive/major project/data.csv')  # Update path as needed

# Step 2: Separate features and target
X = df.drop("label", axis=1, errors='ignore')  # Drop target column
y = df["label"] if 'label' in df.columns else pd.Series([])  # Extract target column

# Encode target labels into numeric values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Check for valid features and labels
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Step 3: Handle categorical and missing data
X = pd.get_dummies(X, drop_first=True)  # One-hot encode categorical variables
X.fillna(X.mean(), inplace=True)  # Fill missing values with column means

Features shape: (238687, 46)
Target shape: (238687,)


In [ ]:
# Step 4: Hybrid Feature Selection
# Filter Method: SelectKBest
filter_selector = SelectKBest(score_func=f_classif, k=8)
X_new_filter = filter_selector.fit_transform(X, y)
filter_scores = filter_selector.scores_

# Wrapper Method: Recursive Feature Elimination (RFE) with XGBoost
rfe_selector = RFE(estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
                   n_features_to_select=7)
X_new_wrapper = rfe_selector.fit_transform(X, y)
rfe_ranking = rfe_selector.ranking_

# Combine results: Select top 10 features from both methods
top_features_filter = np.argsort(filter_scores)[-8:]  # Top 10 from SelectKBest
print('top_features_filter ', top_features_filter)
top_features_rfe = np.where(rfe_ranking == 1)[0]       # Top 10 from RFE
print('top_features_rfe ', top_features_rfe)
top_features_combined = list(set(top_features_filter) | set(top_features_rfe))  # Union
print('top_features_combined ', top_features_combined)

# Step 5: Select top features from the dataset
X_selected = X.iloc[:, top_features_combined]
print('X_selected ', X_selected)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [12 13 22 23 25 28] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:35:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:37:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [06:40:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

 

top_features_filter  [30  7 28 23 13 12 25 22]
top_features_rfe  [ 7  8 10 27 30 34 39]
top_features_combined  [34, 7, 8, 39, 10, 12, 13, 22, 23, 25, 27, 28, 30]
X_selected            Min  fin_flag_number  syn_flag_number           IAT  \
0       54.00              1.0              0.0  8.334383e+07   
1       54.00              0.0              0.0  8.292607e+07   
2       42.00              0.0              0.0  8.312799e+07   
3       50.00              0.0              0.0  8.301570e+07   
4       57.88              0.0              1.0  8.297300e+07   
...       ...              ...              ...           ...   
238682  54.00              1.0              0.0  8.334449e+07   
238683  54.00              0.0              0.0  8.331392e+07   
238684  54.00              0.0              1.0  8.308883e+07   
238685  54.00              0.0              0.0  8.333177e+07   
238686  42.00              0.0              0.0  8.312453e+07   

        psh_flag_number  ece_flag_number  cwr

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Step 6: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

# Step 7: Train XGBoost Classifier
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

# Step 8: Evaluate the model
y_pred = xgb_model.predict(X_test)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
recall = recall_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
f1 = f1_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes

# Print metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Classification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [07:50:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9868169313055987
Precision: 0.9853820613373833
Recall: 0.9868169313055987
F1 Score: 0.9854469726815334
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.77      0.92      0.84      1667
           2       0.00      0.00      0.00         8
           3       0.00      0.00      0.00         6
           4       0.99      0.99      0.99       463
           5       0.65      0.54      0.59        52
           6       1.00      1.00      1.00     11141
           7       1.00      1.00      1.00       685
           8       1.00      1.00      1.00      6319
           9       1.00      1.00      1.00      6238
          10       1.00      1.00      1.00      6261
          11       0.71      0.60      0.65        40
          12       1.00      1.00      1.00      5354
          13       1.00      1.00      1.00      6929
          14       1.00      1.00      1.00      8

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**34 Classes, 10 features**

In [ ]:
# Step 4: Hybrid Feature Selection
# Filter Method: SelectKBest
filter_selector = SelectKBest(score_func=f_classif, k=5)
X_new_filter = filter_selector.fit_transform(X, y)
filter_scores = filter_selector.scores_

# Wrapper Method: Recursive Feature Elimination (RFE) with XGBoost
rfe_selector = RFE(estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
                   n_features_to_select=5)
X_new_wrapper = rfe_selector.fit_transform(X, y)
rfe_ranking = rfe_selector.ranking_

# Combine results: Select top 10 features from both methods
top_features_filter = np.argsort(filter_scores)[-5:]  # Top 10 from SelectKBest
print('top_features_filter ', top_features_filter)
top_features_rfe = np.where(rfe_ranking == 1)[0]       # Top 10 from RFE
print('top_features_rfe ', top_features_rfe)
top_features_combined = list(set(top_features_filter) | set(top_features_rfe))  # Union
print('top_features_combined ', top_features_combined)

# Step 5: Select top features from the dataset
X_selected = X.iloc[:, top_features_combined]
print('X_selected ', X_selected)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [12 13 22 23 25 28] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [07:52:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [07:54:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [07:57:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

 

top_features_filter  [23 13 12 25 22]
top_features_rfe  [ 7  8 10 30 39]
top_features_combined  [39, 8, 7, 10, 12, 13, 22, 23, 25, 30]
X_selected                   IAT  syn_flag_number  fin_flag_number  psh_flag_number  \
0       8.334383e+07              0.0              1.0              0.0   
1       8.292607e+07              0.0              0.0              0.0   
2       8.312799e+07              0.0              0.0              0.0   
3       8.301570e+07              0.0              0.0              0.0   
4       8.297300e+07              1.0              0.0              0.0   
...              ...              ...              ...              ...   
238682  8.334449e+07              0.0              1.0              0.0   
238683  8.331392e+07              0.0              0.0              1.0   
238684  8.308883e+07              1.0              0.0              0.0   
238685  8.333177e+07              0.0              0.0              1.0   
238686  8.312453e+07        

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Step 6: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

# Step 7: Train XGBoost Classifier
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

# Step 8: Evaluate the model
y_pred = xgb_model.predict(X_test)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
recall = recall_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
f1 = f1_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes

# Print metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Classification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:09:54] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9790802575167232
Precision: 0.97985629319059
Recall: 0.9790802575167232
F1 Score: 0.9766624508799187
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.69      0.95      0.80      1667
           2       0.00      0.00      0.00         8
           3       0.00      0.00      0.00         6
           4       1.00      0.98      0.99       463
           5       0.91      0.38      0.54        52
           6       0.99      1.00      1.00     11141
           7       0.99      0.88      0.93       685
           8       1.00      1.00      1.00      6319
           9       1.00      1.00      1.00      6238
          10       1.00      1.00      1.00      6261
          11       1.00      0.05      0.10        40
          12       1.00      1.00      1.00      5354
          13       0.99      1.00      0.99      6929
          14       1.00      1.00      1.00      840

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**34 Classes, 5 features**

In [ ]:
# Step 4: Hybrid Feature Selection
# Filter Method: SelectKBest
filter_selector = SelectKBest(score_func=f_classif, k=3)  # Select top 5 features
X_new_filter = filter_selector.fit_transform(X, y)
filter_scores = filter_selector.scores_

# Wrapper Method: Recursive Feature Elimination (RFE) with XGBoost
rfe_selector = RFE(estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
                   n_features_to_select=2)  # Select top 5 features
X_new_wrapper = rfe_selector.fit_transform(X, y)
rfe_ranking = rfe_selector.ranking_

# Combine results: Select top 15 features from both methods
top_features_filter = np.argsort(filter_scores)[-3:]  # Top 5 from SelectKBest
top_features_rfe = np.where(rfe_ranking == 1)[0]       # Top 5 from RFE
top_features_combined = list(set(top_features_filter) | set(top_features_rfe))  # Union

# Step 5: Select top features from the dataset
X_selected = X.iloc[:, top_features_combined]

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [12 13 22 23 25 28] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:10:54] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:13:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:15:31] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

 

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Step 6: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

# Step 7: Train XGBoost Classifier
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

# Step 8: Evaluate the model
y_pred = xgb_model.predict(X_test)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
recall = recall_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
f1 = f1_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes

# Print metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Classification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:29:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.35979722652813273
Precision: 0.25208462550735744
Recall: 0.35979722652813273
F1 Score: 0.2697649760593752
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.00      0.00      0.00      1667
           2       0.00      0.00      0.00         8
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00       463
           5       0.00      0.00      0.00        52
           6       0.94      1.00      0.97     11141
           7       0.00      0.00      0.00       685
           8       0.00      0.00      0.00      6319
           9       1.00      1.00      1.00      6238
          10       0.00      0.00      0.00      6261
          11       0.00      0.00      0.00        40
          12       0.00      0.00      0.00      5354
          13       0.00      0.00      0.00      6929
          14       0.16      1.00      0.27    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
